In [1]:
from sqlalchemy import create_engine

# Import pandas, numpy, matplotlib, seaborn libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import datetime

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn import metrics

In [2]:
%store -r data

In [3]:
#Retrieve the data of corresponding line of bus
list_id15=df_GTFS['trip_id_new'].loc[df_GTFS['LineId']==data].unique().tolist()
df_GTFS_15 = df_GTFS[df_GTFS['trip_id_new'].isin(list_id15)]

#we save the unique departure time in df_GTFS_15
df_GTFS_15_deptime_list = df_GTFS_15['departure_time'].unique().tolist()
df_stoptime_15 = df_stoptimes_new[df_stoptimes_new['trip_id_new'].isin(list_id15)]

#we will save the trip_id for the departure_time matches in the df_GTFS_15_deptime_list
trip_id_list = df_stoptime_15['trip_id'].loc[df_stoptime_15['departure_time'].isin(df_GTFS_15_deptime_list)].tolist()

#We create column 'StartDate' in df_stoptime_15
df_stoptime_15['StartDate'] = ''
df_GTFS_15['StartDate']=df_GTFS_15['StartDate'].astype(str)

#we add merge StartDate to the 'df_stoptime_15'
list_for_concat = []
for i in trip_id_list:
    start_timevalue = df_stoptime_15['departure_time'].loc[(df_stoptime_15['trip_id']==i)&(df_stoptime_15['stop_sequence']==1)].values[0]
    trip_id_new_value = df_stoptime_15['trip_id_new'].loc[(df_stoptime_15['trip_id']==i)&(df_stoptime_15['stop_sequence']==1)].values[0]
    startdate_list = df_GTFS_15['StartDate'].loc[(df_GTFS_15['departure_time']==start_timevalue)&(df_GTFS_15['trip_id_new']==trip_id_new_value)].unique().tolist()
    #print(startdate_list)
    
    for j in startdate_list:
        df_stoptime_15['StartDate'].loc[(df_stoptime_15['trip_id']==i)] = j
        list_for_concat.append(df_stoptime_15.loc[df_stoptime_15['StartDate']==j])

if len(list_for_concat)==0:
    print('None date in GTFS could match with the data in stop_times')
else:
    df_stoptime_15_new = pd.concat(list_for_concat)

    #Now we will merger the data of df_stoptime_15_new and df_GTFS_15
    df_stoptimes_merged = pd.merge(df_stoptime_15_new,df_GTFS_15[['trip_id_new','StartDate','stop_sequence','stop_id','Arr_delay','Dep_delay','LineId']],how="left", on=['trip_id_new','StartDate','stop_sequence','stop_id'])
    df_stoptimes_merged['Arr_delay'].fillna(0, inplace=True)
    df_stoptimes_merged['Dep_delay'].fillna(0, inplace=True)
    df_stoptimes_merged['LineId'].fillna(15, inplace=True)

    #convert the data type of time into datetime
    df_stoptimes_merged['StartDate']=df_stoptimes_merged['StartDate'].astype('datetime64')
    df_stoptimes_merged['date_of_service'] = df_stoptimes_merged['StartDate'].dt.strftime("%Y-%m-%d")
    df_stoptimes_merged['arrival_time'] = df_stoptimes_merged['date_of_service'] + str(' ') + df_stoptimes_merged['arrival_time']
    df_stoptimes_merged['departure_time'] = df_stoptimes_merged['date_of_service'] + str(' ') + df_stoptimes_merged['departure_time']

    df_stoptimes_merged['arrival_time'] = df_stoptimes_merged['arrival_time'].astype('datetime64')
    df_stoptimes_merged['departure_time'] = df_stoptimes_merged['departure_time'].astype('datetime64')

    #calcuate the actual depart time and actual arrive time
    df_stoptimes_merged['actual_arrival_time'] = 0
    df_stoptimes_merged['actual_departure_time'] = 0
    for i in range(len(df_stoptimes_merged)):
        df_stoptimes_merged['actual_arrival_time'][i] = df_stoptimes_merged['arrival_time'][i] + pd.Timedelta(seconds=df_stoptimes_merged['Arr_delay'][i])
        df_stoptimes_merged['actual_departure_time'][i] = df_stoptimes_merged['departure_time'][i] + pd.Timedelta(seconds=df_stoptimes_merged['Dep_delay'][i])
    print('Finished with calculating actual depart time and actual arrive time')

    #convert the data type of time into datetime
    df_stoptimes_merged['actual_arrival_time'] = df_stoptimes_merged['actual_arrival_time'].astype('datetime64')
    df_stoptimes_merged['actual_departure_time'] = df_stoptimes_merged['actual_departure_time'].astype('datetime64')

    #Calculate the accumulated travel time
    df_stoptimes_merged['acc_travel_time'] = (df_stoptimes_merged['actual_arrival_time'] - df_stoptimes_merged['StartDate'])
    df_stoptimes_merged['acc_travel_time'] = df_stoptimes_merged['acc_travel_time'].dt.total_seconds()

    #Create the following new features for modeling
    df_stoptimes_merged['hour_of_day']=df_stoptimes_merged['actual_arrival_time'].dt.hour
    df_stoptimes_merged['day_of_week']=df_stoptimes_merged['actual_arrival_time'].dt.dayofweek
    df_stoptimes_merged['month_of_year']=df_stoptimes_merged['actual_arrival_time'].dt.month

    #Prepare data for merging with weather info
    df_stoptimes_merged['date']=df_stoptimes_merged['actual_arrival_time'].astype('datetime64[h]')
    df_stoptimes_merged=pd.merge(df_stoptimes_merged,weather[['date','temp','rain']],how="left",on=['date'])

    #Clean up the columns
    df_stoptimes_merged_cleaned = df_stoptimes_merged[['trip_id','arrival_time','stop_id','stop_sequence','stop_headsign','trip_id_new','StartDate','LineId','actual_arrival_time','actual_departure_time','acc_travel_time','hour_of_day','day_of_week','month_of_year','temp','rain']]
    df_stoptimes_merged_cleaned['temp'].fillna(0, inplace=True)
    df_stoptimes_merged_cleaned['rain'].fillna(0, inplace=True)
    df_stoptimes_merged_cleaned.to_csv('df_stoptimes_route_{}.csv'.format(lineid),index=False)
    
    if len(df_stoptimes_merged_cleaned['stop_headsign'].unique().tolist()) == 1:
        print('Only one headsign or Nan value in headsign column.')

        # #Select the features we need for modeling and Copy the original version of dataset
        df_features_for_ML1 = df_stoptimes_merged_cleaned[['stop_sequence','acc_travel_time','hour_of_day','day_of_week','month_of_year','temp','rain']]
        # df_features_for_ML['stop_headsign'].fillna('n/a',inplace=True)
        # df_features_for_ML['temp'].fillna(0, inplace=True)
        # df_features_for_ML['rain'].fillna(0, inplace=True)
        # df_features_for_ML['stop_headsign'].astype('category')

        # # set up dummies features
        # df_features_for_ML = pd.get_dummies(df_features_for_ML, drop_first=False)

        #Setting up the train/test split
        # y is the target
        y = df_features_for_ML1["acc_travel_time"]
        # X is everything else
        X = df_features_for_ML1.drop(["acc_travel_time"],1)
        # Split the dataset into two datasets: 70% training and 30% test
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,  test_size=0.3)


        # Train RF with 10 trees
        rfc = RandomForestRegressor(n_estimators=10, max_features='auto', oob_score=True, random_state=1).fit(X_train, y_train)

        #Predict the test dataset
        rfc_predictions_test = rfc.predict(X_test)

        #Calculate the metrics
        MAE_rfc = metrics.mean_absolute_error(y_test, rfc_predictions_test)
        RMSE_rfc = metrics.mean_squared_error(y_test, rfc_predictions_test)**0.5
        R2_rfc = metrics.r2_score(y_test, rfc_predictions_test)

        # writing these evaluation scores to a file
        f = open("GTFS_RandomForest_Evaluation_08_08.txt", "a")
        f.write("\nLine: " + str(data)  + ". MAE: " + str(MAE_rfc) + ". RMSE: " + str(RMSE_rfc) + ". R2 Score: " + str(R2_rfc) + ".\n")
        f.close()

        # save the model to pickle file
        filename = "ModelResult_RandomForest_GTFS/PickleFiles_{}.pkl".format(lineid)
        pickle.dump(rfc, open(filename, 'wb'))
    
    elif len(df_stoptimes_merged_cleaned['stop_headsign'].unique().tolist()) > 1:
        print('Length of headsign: ', len(df_stoptimes_merged_cleaned['stop_headsign'].unique().tolist()))

        headsign1 = df_stoptimes_merged_cleaned['stop_headsign'].unique().tolist()[0]
        headsign2 = df_stoptimes_merged_cleaned['stop_headsign'].unique().tolist()[1]

        #Select the features we need for modeling and Copy the original version of dataset
        df_features_for_ML1 = df_stoptimes_merged_cleaned[['stop_sequence','acc_travel_time','hour_of_day','day_of_week','month_of_year','temp','rain']].loc[df_stoptimes_merged_cleaned['stop_headsign']==headsign1]
        df_features_for_ML2 = df_stoptimes_merged_cleaned[['stop_sequence','acc_travel_time','hour_of_day','day_of_week','month_of_year','temp','rain']].loc[df_stoptimes_merged_cleaned['stop_headsign']==headsign2]
        # df_features_for_ML['stop_headsign'].fillna('n/a',inplace=True)
        # df_features_for_ML['temp'].fillna(0, inplace=True)
        # df_features_for_ML['rain'].fillna(0, inplace=True)
        # df_features_for_ML['stop_headsign'].astype('category')

        # # set up dummies features
        # df_features_for_ML = pd.get_dummies(df_features_for_ML, drop_first=False)

        #Setting up the train/test split
        # y is the target
        y = df_features_for_ML1["acc_travel_time"]
        # X is everything else
        X = df_features_for_ML1.drop(["acc_travel_time"],1)
        # Split the dataset into two datasets: 70% training and 30% test
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,  test_size=0.3)


        # Train RF with 10 trees
        rfc = RandomForestRegressor(n_estimators=10, max_features='auto', oob_score=True, random_state=1).fit(X_train, y_train)

        #Predict the test dataset
        rfc_predictions_test = rfc.predict(X_test)

        #Calculate the metrics
        MAE_rfc = metrics.mean_absolute_error(y_test, rfc_predictions_test)
        RMSE_rfc = metrics.mean_squared_error(y_test, rfc_predictions_test)**0.5
        R2_rfc = metrics.r2_score(y_test, rfc_predictions_test)

        headsign1 = headsign1.replace(' ','')
        # writing these evaluation scores to a file
        f = open("GTFS_RandomForest_Evaluation_08_08.txt", "a")
        f.write("\nLine: " + str(data) + '. Head sign stop: ' + headsign1 + ". MAE: " + str(MAE_rfc) + ". RMSE: " + str(RMSE_rfc) + ". R2 Score: " + str(R2_rfc) + ".\n")
        f.close()

        # save the model to pickle file
        filename = "ModelResult_RandomForest_GTFS/PickleFiles_{}_{}.pkl".format(lineid,headsign1)
        pickle.dump(rfc, open(filename, 'wb'))

        #Setting up the train/test split
        # y is the target
        y = df_features_for_ML2["acc_travel_time"]
        # X is everything else
        X = df_features_for_ML2.drop(["acc_travel_time"],1)
        # Split the dataset into two datasets: 70% training and 30% test
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,  test_size=0.3)


        # Train RF with 10 trees
        rfc = RandomForestRegressor(n_estimators=10, max_features='auto', oob_score=True, random_state=1).fit(X_train, y_train)

        #Predict the test dataset
        rfc_predictions_test = rfc.predict(X_test)

        #Calculate the metrics
        MAE_rfc = metrics.mean_absolute_error(y_test, rfc_predictions_test)
        RMSE_rfc = metrics.mean_squared_error(y_test, rfc_predictions_test)**0.5
        R2_rfc = metrics.r2_score(y_test, rfc_predictions_test)

        headsign2 = headsign2.replace(' ','')
        # writing these evaluation scores to a file
        f = open("GTFS_RandomForest_Evaluation_08_08.txt", "a")
        f.write("\nLine: " + str(data) + '. Head sign stop: ' + headsign2 + ". MAE: " + str(MAE_rfc) + ". RMSE: " + str(RMSE_rfc) + ". R2 Score: " + str(R2_rfc) + ".\n")
        f.close()

        # save the model to pickle file
        filename = "ModelResult_RandomForest_GTFS/PickleFiles_{}_{}.pkl".format(lineid,headsign2)
        pickle.dump(rfc, open(filename, 'wb'))
    
    
    
#     #Select the features we need for modeling and Copy the original version of dataset
#     df_features_for_ML = df_stoptimes_merged_cleaned[['stop_sequence','stop_headsign','acc_travel_time','hour_of_day','day_of_week','month_of_year','temp','rain']]
#     df_features_for_ML['stop_headsign'].astype('category')

#     # set up dummies features
#     df_features_for_ML = pd.get_dummies(df_features_for_ML, drop_first=False)


#     #Setting up the train/test split
#     # y is the target
#     y = df_features_for_ML["acc_travel_time"]
#     # X is everything else
#     X = df_features_for_ML.drop(["acc_travel_time"],1)
#     # Split the dataset into two datasets: 70% training and 30% test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,  test_size=0.3)


#     # Train aka fit, a model using all continuous and categorical features.
#     linreg = LinearRegression().fit(X_train, y_train)

#     #Predict the test dataset
#     linreg_predictions_test = linreg.predict(X_test)

#     #Calculate the metrics
#     MAE_linreg = metrics.mean_absolute_error(y_test, linreg_predictions_test)
#     RMSE_linreg = metrics.mean_squared_error(y_test, linreg_predictions_test)**0.5
#     R2_linreg = metrics.r2_score(y_test, linreg_predictions_test)

#     # writing these evaluation scores to a file
#     f = open("GTFS_LinearRegres_Evaluation_08_08.txt", "a")
#     f.write("\nLine: " + str(data) + ". MAE: " + str(MAE_linreg) + ". RMSE: " + str(RMSE_linreg) + ". R2 Score: " + str(R2_linreg) + ".\n")
#     f.close()

#     # save the model to pickle file
#     filename = "ModelResult_LinearRegression_GTFS/PickleFiles_{}.pkl".format(lineid)
#     pickle.dump(linreg, open(filename, 'wb'))


#     # Train RF with 10 trees
#     rfc = RandomForestRegressor(n_estimators=10, max_features='auto', oob_score=True, random_state=1).fit(X_train, y_train)

#     #Predict the test dataset
#     rfc_predictions_test = rfc.predict(X_test)

#     #Calculate the metrics
#     MAE_rfc = metrics.mean_absolute_error(y_test, rfc_predictions_test)
#     RMSE_rfc = metrics.mean_squared_error(y_test, rfc_predictions_test)**0.5
#     R2_rfc = metrics.r2_score(y_test, rfc_predictions_test)

#     # writing these evaluation scores to a file
#     f = open("GTFS_RandomForest_Evaluation_08_08.txt", "a")
#     f.write("\nLine: " + str(data) + ". MAE: " + str(MAE_rfc) + ". RMSE: " + str(RMSE_rfc) + ". R2 Score: " + str(R2_rfc) + ".\n")
#     f.close()

#     # save the model to pickle file
#     filename = "ModelResult_RandomForest_GTFS/PickleFiles_{}.pkl".format(lineid)
#     pickle.dump(linreg, open(filename, 'wb'))

In [2]:
#df_stoptimes_merged_cleaned = pd.read_csv('df_stoptimes_route_{}.csv'.format(lineid))

# if len(df_stoptimes_merged_cleaned['stop_headsign'].unique().tolist()) == 1:
#     print('Only one headsign or Nan value in headsign column.')
    
#     # #Select the features we need for modeling and Copy the original version of dataset
#     df_features_for_ML1 = df_stoptimes_merged_cleaned[['stop_sequence','acc_travel_time','hour_of_day','day_of_week','month_of_year','temp','rain']]
#     # df_features_for_ML['stop_headsign'].fillna('n/a',inplace=True)
#     # df_features_for_ML['temp'].fillna(0, inplace=True)
#     # df_features_for_ML['rain'].fillna(0, inplace=True)
#     # df_features_for_ML['stop_headsign'].astype('category')

#     # # set up dummies features
#     # df_features_for_ML = pd.get_dummies(df_features_for_ML, drop_first=False)

#     #Setting up the train/test split
#     # y is the target
#     y = df_features_for_ML1["acc_travel_time"]
#     # X is everything else
#     X = df_features_for_ML1.drop(["acc_travel_time"],1)
#     # Split the dataset into two datasets: 70% training and 30% test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,  test_size=0.3)


#     # Train RF with 10 trees
#     rfc = RandomForestRegressor(n_estimators=10, max_features='auto', oob_score=True, random_state=1).fit(X_train, y_train)

#     #Predict the test dataset
#     rfc_predictions_test = rfc.predict(X_test)

#     #Calculate the metrics
#     MAE_rfc = metrics.mean_absolute_error(y_test, rfc_predictions_test)
#     RMSE_rfc = metrics.mean_squared_error(y_test, rfc_predictions_test)**0.5
#     R2_rfc = metrics.r2_score(y_test, rfc_predictions_test)

#     # writing these evaluation scores to a file
#     f = open("GTFS_RandomForest_Evaluation_08_08.txt", "a")
#     f.write("\nLine: " + str(data)  + ". MAE: " + str(MAE_rfc) + ". RMSE: " + str(RMSE_rfc) + ". R2 Score: " + str(R2_rfc) + ".\n")
#     f.close()

#     # save the model to pickle file
#     filename = "ModelResult_RandomForest_GTFS/PickleFiles_{}.pkl".format(lineid)
#     pickle.dump(rfc, open(filename, 'wb'))
    
# elif len(df_stoptimes_merged_cleaned['stop_headsign'].unique().tolist()) > 1:
#     print('Length of headsign: ', len(df_stoptimes_merged_cleaned['stop_headsign'].unique().tolist()))

#     headsign1 = df_stoptimes_merged_cleaned['stop_headsign'].unique().tolist()[0]
#     headsign2 = df_stoptimes_merged_cleaned['stop_headsign'].unique().tolist()[1]

#     #Select the features we need for modeling and Copy the original version of dataset
#     df_features_for_ML1 = df_stoptimes_merged_cleaned[['stop_sequence','acc_travel_time','hour_of_day','day_of_week','month_of_year','temp','rain']].loc[df_stoptimes_merged_cleaned['stop_headsign']==headsign1]
#     df_features_for_ML2 = df_stoptimes_merged_cleaned[['stop_sequence','acc_travel_time','hour_of_day','day_of_week','month_of_year','temp','rain']].loc[df_stoptimes_merged_cleaned['stop_headsign']==headsign2]
#     # df_features_for_ML['stop_headsign'].fillna('n/a',inplace=True)
#     # df_features_for_ML['temp'].fillna(0, inplace=True)
#     # df_features_for_ML['rain'].fillna(0, inplace=True)
#     # df_features_for_ML['stop_headsign'].astype('category')

#     # # set up dummies features
#     # df_features_for_ML = pd.get_dummies(df_features_for_ML, drop_first=False)

#     #Setting up the train/test split
#     # y is the target
#     y = df_features_for_ML1["acc_travel_time"]
#     # X is everything else
#     X = df_features_for_ML1.drop(["acc_travel_time"],1)
#     # Split the dataset into two datasets: 70% training and 30% test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,  test_size=0.3)


#     # Train RF with 10 trees
#     rfc = RandomForestRegressor(n_estimators=10, max_features='auto', oob_score=True, random_state=1).fit(X_train, y_train)

#     #Predict the test dataset
#     rfc_predictions_test = rfc.predict(X_test)

#     #Calculate the metrics
#     MAE_rfc = metrics.mean_absolute_error(y_test, rfc_predictions_test)
#     RMSE_rfc = metrics.mean_squared_error(y_test, rfc_predictions_test)**0.5
#     R2_rfc = metrics.r2_score(y_test, rfc_predictions_test)

#     headsign1 = headsign1.replace(' ','')
#     # writing these evaluation scores to a file
#     f = open("GTFS_RandomForest_Evaluation_08_08.txt", "a")
#     f.write("\nLine: " + str(data) + '. Head sign stop: ' + headsign1 + ". MAE: " + str(MAE_rfc) + ". RMSE: " + str(RMSE_rfc) + ". R2 Score: " + str(R2_rfc) + ".\n")
#     f.close()

#     # save the model to pickle file
#     filename = "ModelResult_RandomForest_GTFS/PickleFiles_{}_{}.pkl".format(lineid,headsign1)
#     pickle.dump(rfc, open(filename, 'wb'))

#     #Setting up the train/test split
#     # y is the target
#     y = df_features_for_ML2["acc_travel_time"]
#     # X is everything else
#     X = df_features_for_ML2.drop(["acc_travel_time"],1)
#     # Split the dataset into two datasets: 70% training and 30% test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,  test_size=0.3)


#     # Train RF with 10 trees
#     rfc = RandomForestRegressor(n_estimators=10, max_features='auto', oob_score=True, random_state=1).fit(X_train, y_train)

#     #Predict the test dataset
#     rfc_predictions_test = rfc.predict(X_test)

#     #Calculate the metrics
#     MAE_rfc = metrics.mean_absolute_error(y_test, rfc_predictions_test)
#     RMSE_rfc = metrics.mean_squared_error(y_test, rfc_predictions_test)**0.5
#     R2_rfc = metrics.r2_score(y_test, rfc_predictions_test)

#     headsign2 = headsign2.replace(' ','')
#     # writing these evaluation scores to a file
#     f = open("GTFS_RandomForest_Evaluation_08_08.txt", "a")
#     f.write("\nLine: " + str(data) + '. Head sign stop: ' + headsign2 + ". MAE: " + str(MAE_rfc) + ". RMSE: " + str(RMSE_rfc) + ". R2 Score: " + str(R2_rfc) + ".\n")
#     f.close()

#     # save the model to pickle file
#     filename = "ModelResult_RandomForest_GTFS/PickleFiles_{}_{}.pkl".format(lineid,headsign2)
#     pickle.dump(rfc, open(filename, 'wb'))

Length of headsign:  2


ValueError: With n_samples=0, test_size=0.2 and train_size=0.8, the resulting train set will be empty. Adjust any of the aforementioned parameters.